# Tutorial: Row Level Security

This notebook uses the sample AdventureWorks sample database but can be used with any user database.

# Step 1: Understanding the Hierarchy in AdventureWorks

## In this step, we will navigate through the database to understand the business requirements for the row level security. 

## 1.  I have populated the database logins principals with the users that are part of the hierarchy.

In [ ]:
Use  AdventureWorks
Go
CREATE USER ken0 WITH PASSWORD = 'K3nk3nk3n';
ALTER ROLE db_datareader ADD MEMBER ken0;
ALTER ROLE db_datawriter ADD MEMBER ken0;

CREATE USER brian3 WITH PASSWORD = 'Br1@n3br1@n3';
ALTER ROLE db_datareader ADD MEMBER brian3;
ALTER ROLE db_datawriter ADD MEMBER brian3;

CREATE USER amy0 WITH PASSWORD = 'Am10@m10';
ALTER ROLE db_datareader ADD MEMBER amy0;
ALTER ROLE db_datawriter ADD MEMBER amy0;

CREATE USER jae0 WITH PASSWORD = 'Ja3ja3ja3';
ALTER ROLE db_datareader ADD MEMBER jae0;
ALTER ROLE db_datawriter ADD MEMBER jae0;

CREATE USER rachel0 WITH PASSWORD = 'Rach3lrach3l';
ALTER ROLE db_datareader ADD MEMBER rachel0;
ALTER ROLE db_datawriter ADD MEMBER rachel0;

CREATE USER ranjit0 WITH PASSWORD = 'Ranj1tranj1t';
ALTER ROLE db_datareader ADD MEMBER ranjit0;
ALTER ROLE db_datawriter ADD MEMBER ranjit0;


# Step 2: Let's review the hierarchy

In [ ]:
Use  AdventureWorks
Go
SELECT e.[BusinessEntityID]
	  ,e.[LoginID]
      ,e.[OrganizationLevel]
      ,e.[JobTitle]
	  ,m.LoginID as manager
  FROM [AdventureWorks].[HumanResources].[Employee] e
  inner join sys.database_principals dp  on 'adventure-works\'+dp.name COLLATE SQL_Latin1_General_CP1_CI_AS= e.loginid
  Left outer JOIN [HumanResources].[Employee] m on e.OrganizationNode.GetAncestor(1)=m.OrganizationNode
  order by e.[OrganizationLevel], e.LoginID
  


# Step 3: Let's check on the Sales Orders for the European Sales Manager

In [ ]:
Use  AdventureWorks
Go
Execute as user = 'amy0'
Select e.LoginID
	,Count(*) TotalOrders
	, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
INNER JOIN [HumanResources].[Employee] e on (s.SalesPersonID= e.BusinessEntityID)
INNER JOIN [HumanResources].[Employee] m on e.OrganizationNode.IsDescendantOf(m.OrganizationNode)=1  --- This will bring Amy's team's orders as well as hers
where m.LoginID='adventure-works\'+User_name() COLLATE SQL_Latin1_General_CP1_CI_AS
group by  e.LoginID WITH ROLLUP
REVERT


# Step 4: Now, let's have a look at Jae's orders

In [ ]:
Use  AdventureWorks
Go
Execute as user = 'jae0'
Select e.LoginID, Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
INNER JOIN [HumanResources].[Employee] e on (s.SalesPersonID= e.BusinessEntityID)
INNER JOIN [HumanResources].[Employee] m on e.OrganizationNode.IsDescendantOf(m.OrganizationNode)=1
where m.LoginID='adventure-works\'+User_name() COLLATE SQL_Latin1_General_CP1_CI_AS
Group by e.LoginID
REVERT

# Step 5: But Jae can see all the orders, not only hers.

In [ ]:
Use  AdventureWorks
Go
Execute as user = 'jae0'
Select e.LoginID
	,Count(*) TotalOrders
	, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
INNER JOIN [HumanResources].[Employee] e on (s.SalesPersonID= e.BusinessEntityID)
group by  e.LoginID
REVERT

We have two Business requirements for the row level security:

- Sales Representatives can only see their data.
- Managers can see their direct reports data so in other words if we have two sales people who report to the same person or report to different managers even they cannot see each other's data they can only see their own whereas a manager can see the data for the entire team. If the VP of Sales, Brian (brian3) is online and querying the orders we will aso account for the online orders which have no Sales Person ID.

The first requirement can be fulfilled by the query below:

In [ ]:
Use  AdventureWorks
Go
Execute as user = 'jae0'
Select e.LoginID, Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
INNER JOIN [HumanResources].[Employee] e on (s.SalesPersonID= e.BusinessEntityID)
where e.LoginID='adventure-works\'+User_name() COLLATE SQL_Latin1_General_CP1_CI_AS
Group by e.LoginID
REVERT

The second requirement can be fulfilled by the query below:

In [ ]:
Use  AdventureWorks
Go
Execute as user = 'amy0'
Select e.LoginID	,Count(*) TotalOrders 	, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
INNER JOIN [HumanResources].[Employee] e on (s.SalesPersonID= e.BusinessEntityID)
INNER JOIN [HumanResources].[Employee] m on e.OrganizationNode.IsDescendantOf(m.OrganizationNode)=1  --- This will bring Amy's team's orders as well as hers
where m.LoginID='adventure-works\'+User_name() COLLATE SQL_Latin1_General_CP1_CI_AS
group by  e.LoginID
REVERT


If we execute the above query for jae0, as jae0 has no one reporting to her it will not affect the result.

Let's add the logic for the VP of Sales and create a user defined function.

# Step 6: Create a user defined function to hold this business requirements.

As part of the best practices to maintain the row level security policies, we will create a schema to hold the function

In [ ]:
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'RowLevelSecurity')
BEGIN
    EXEC('CREATE SCHEMA RowLevelSecurity')
END
Go
CREATE OR ALTER FUNCTION RowLevelSecurity.udf_BusinessEntityID (@BusinessEntityID int)
RETURNS TABLE WITH SCHEMABINDING
AS 
RETURN (
	Select 1 Result
    From Sales.SalesOrderHeader as s 
    INNER JOIN [HumanResources].[Employee] e on (s.SalesPersonID= e.BusinessEntityID)
    INNER JOIN [HumanResources].[Employee] m on e.OrganizationNode.IsDescendantOf(m.OrganizationNode)=1
    where e.BusinessEntityID=@BusinessEntityID and m.LoginID='adventure-works\'+User_name() COLLATE SQL_Latin1_General_CP1_CI_AS
    UNION
    --- Return all online orders for the VP of Sales when he is online
    Select 1 Result
    From Sales.SalesOrderHeader as s 
    CROSS JOIN [HumanResources].[Employee] e 
    WHERE e.LoginID='adventure-works\'+User_name() COLLATE SQL_Latin1_General_CP1_CI_AS  --- If VP of Sales is querying
    AND e.OrganizationLevel=1 ----VP of Sales
    and s.OnlineOrderFlag=1 ---- online orders
	
)

Let's run the queries for using the function now.

In [ ]:
--- Now testing the function 
Execute as user = 'jae0'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
CROSS APPLY RowLevelSecurity.udf_BusinessEntityID(s.SalesPersonID)  p
REVERT

Execute as user = 'rachel0'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
CROSS APPLY RowLevelSecurity.udf_BusinessEntityID(s.SalesPersonID)  p
REVERT

Execute as user = 'ranjit0'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
CROSS APPLY RowLevelSecurity.udf_BusinessEntityID(s.SalesPersonID)  p
REVERT

Execute as user = 'amy0'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
CROSS APPLY RowLevelSecurity.udf_BusinessEntityID(s.SalesPersonID)  p
REVERT

Execute as user = 'brian3'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
CROSS APPLY RowLevelSecurity.udf_BusinessEntityID(s.SalesPersonID)  p
REVERT


# Step 6: Create a user defined function to hold this business requirements.

As part of the best practices to maintain the row level security policies, we will create a schema to hold the function

In [ ]:

-- Check if the security policy exists
IF EXISTS (SELECT * FROM sys.security_policies WHERE name = 'secpolBusinessEntityID')
BEGIN
    -- Drop the existing security policy
    DROP SECURITY POLICY secpolBusinessEntityID
END

-- Create or alter the security policy with a filter predicate
CREATE SECURITY POLICY RowLevelSecurity.secpolBusinessEntityID
ADD FILTER PREDICATE RowLevelSecurity.udf_BusinessEntityID(SalesPersonID) ON Sales.SalesOrderHeader
GO

/*
--- Recreate it adding block predicate
DROP SECURITY POLICY RowLevelSecurity.secpolBusinessEntityID
GO
CREATE SECURITY POLICY RowLevelSecurity.secpolBusinessEntityID
ADD FILTER PREDICATE RowLevelSecurity.udf_BusinessEntityID(SalesPersonID) ON Sales.SalesOrderHeader,
ADD BLOCK PREDICATE RowLevelSecurity.udf_BusinessEntityID(SalesPersonID) ON Sales.SalesOrderHeader   ---- NO CRUD
*/

Let's test the security policy.

In [ ]:

--- Now testing the row level security policy 
Execute as user = 'jae0'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
REVERT

Execute as user = 'rachel0'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
REVERT

Execute as user = 'ranjit0'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
REVERT

Execute as user = 'amy0'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
REVERT

Execute as user = 'brian3'
Select Count(*) TotalOrders, Sum(SubTotal) SubTotal
From Sales.SalesOrderHeader as s 
REVERT
